# RoleColorAI - Resume Intelligence Demo

This notebook demonstrates the RoleColorAI system that analyzes and rewrites resumes through a team-role lens.

## Setup

In [ ]:
# Install dependencies if needed
# !pip install -r requirements.txt

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Import our modules
from rolecolor_framework import ROLECOLOR_KEYWORDS, get_rolecolor_descriptions
from resume_scorer import score_resume, format_score_output
from resume_rewriter import rewrite_summary
from latex_generator import generate_simple_latex, generate_latex_from_resume

print("✅ All modules loaded successfully!")

## Part 1: RoleColor Keyword Framework

The framework defines weighted keywords for each of the four RoleColors.

In [ ]:
# Display keyword framework
print("=" * 60)
print("ROLECOLOR KEYWORD FRAMEWORK")
print("=" * 60)

descriptions = get_rolecolor_descriptions()

for role, keywords in ROLECOLOR_KEYWORDS.items():
    print(f"\n🔹 {role.upper()} ({len(keywords)} keywords)")
    print("-" * 40)
    print(f"Description: {descriptions[role]}")
    print("\nTop keywords (by weight):")
    
    # Show top 8 keywords by weight
    sorted_kw = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:8]
    for kw, weight in sorted_kw:
        print(f"  • {kw}: {weight}")

### Keyword Selection Rationale

| RoleColor | Keyword Focus | Examples | Why? |
|-----------|--------------|----------|------|
| **Builder** | Strategic & innovation language | strategy, vision, architect, scalable | Builders drive long-term thinking and create foundations |
| **Enabler** | Collaborative & bridging language | collaborate, mentor, coordinate, facilitate | Enablers connect people and ensure smooth communication |
| **Thriver** | Speed & ownership language | deadline, fast-paced, ownership, deliver | Thrivers excel under pressure and take accountability |
| **Supportee** | Reliability & process language | maintain, document, reliable, compliance | Supportees ensure consistency and operational stability |

## Part 2: Resume Scoring

Let's score some sample resumes and see their RoleColor distributions.

In [ ]:
# Sample Resume 1: Builder-leaning
resume_builder = """
Senior Software Engineer with 5 years of experience building scalable 
backend systems. Led the architecture and development of a microservices 
platform handling 10M+ daily transactions.

Key Achievements:
- Architected a real-time data processing pipeline using Kafka and Spark
- Designed and implemented innovative caching solutions
- Spearheaded the cloud migration strategy, transforming infrastructure
- Created scalable API frameworks adopted across the organization
- Pioneered adoption of new technologies improving system performance by 60%
"""

result = score_resume(resume_builder)
print(format_score_output(result))

In [ ]:
# Sample Resume 2: Enabler-leaning
resume_enabler = """
Project Manager with 6 years of experience facilitating cross-functional 
collaboration and delivering complex software projects.

Key Achievements:
- Coordinated between engineering, design, and business stakeholders
- Mentored 5 junior project managers on agile methodologies
- Facilitated weekly cross-functional meetings improving team alignment
- Collaborated with clients to gather requirements and manage expectations
- Enabled smooth communication between technical and non-technical teams
- Partnered with executive leadership to align goals
"""

result = score_resume(resume_enabler)
print(format_score_output(result))

In [ ]:
# Sample Resume 3: Supportee-leaning
resume_supportee = """
DevOps Engineer with 4 years of experience ensuring system reliability 
and maintaining production infrastructure with 99.99% uptime.

Key Achievements:
- Maintained and monitored production systems handling critical transactions
- Documented all operational procedures and runbooks for the team
- Ensured compliance with SOC2 and PCI-DSS security standards
- Established systematic approach to incident response
- Maintained reliable CI/CD pipelines with consistent deployment success
- Performed regular audits of security configurations
"""

result = score_resume(resume_supportee)
print(format_score_output(result))

## Part 3: Resume Summary Rewriting

Using OpenAI to generate role-aligned summaries based on the dominant RoleColor.

In [ ]:
# Input resume to rewrite
input_resume = """
Software engineer with 2 years of experience in backend development and APIs.
Worked on Python and Node.js projects. Built REST APIs and worked with databases.
Participated in code reviews and helped debug production issues.
"""

# First, score the resume
scores_result = score_resume(input_resume)
print("Original Resume:")
print(input_resume)
print("\n" + "=" * 50)
print("Score Analysis:")
print("=" * 50)
for role, score in sorted(scores_result['scores'].items(), key=lambda x: x[1], reverse=True):
    print(f"{role}: {score:.2%}")
print(f"\nDominant RoleColor: {scores_result['dominant_role']}")

In [ ]:
# Generate rewritten summary
print("=" * 50)
print("REWRITTEN SUMMARY (Builder-focused)")
print("=" * 50)

rewritten = rewrite_summary(
    resume_text=input_resume,
    dominant_role="Builder",
    scores=scores_result['scores']
)

print(rewritten)

In [ ]:
# Generate for different RoleColors
for role in ["Enabler", "Thriver", "Supportee"]:
    print("\n" + "=" * 50)
    print(f"REWRITTEN SUMMARY ({role}-focused)")
    print("=" * 50)
    
    rewritten = rewrite_summary(
        resume_text=input_resume,
        dominant_role=role,
        scores=scores_result['scores']
    )
    
    print(rewritten)

## Bonus: LaTeX Generation

Generate professional LaTeX code for Overleaf.

In [ ]:
# Generate simple LaTeX template
latex_code = generate_simple_latex(
    name="John Doe",
    contact="john.doe@email.com | (555) 123-4567 | New York, NY",
    summary=rewritten,
    dominant_role="Builder"
)

print("Generated LaTeX Code:")
print("=" * 50)
print(latex_code)

## Complete Example: End-to-End Pipeline

In [ ]:
def analyze_resume(resume_text: str):
    """
    Complete end-to-end resume analysis.
    
    Args:
        resume_text: The resume text to analyze
        
    Returns:
        Dictionary with scores, summary, and analysis
    """
    # Step 1: Score the resume
    print("📊 Analyzing RoleColor distribution...")
    scores = score_resume(resume_text)
    
    print(f"\n✅ Dominant RoleColor: {scores['dominant_role']}")
    print("\nScore Distribution:")
    for role, score in sorted(scores['scores'].items(), key=lambda x: x[1], reverse=True):
        bar = "█" * int(score * 30) + "░" * (30 - int(score * 30))
        print(f"  {role:10} [{bar}] {score:.1%}")
    
    # Step 2: Generate rewritten summary
    print("\n✍️ Generating role-aligned summary...")
    summary = rewrite_summary(
        resume_text=resume_text,
        dominant_role=scores['dominant_role'],
        scores=scores['scores']
    )
    
    print(f"\n📝 Rewritten Summary ({scores['dominant_role']}-focused):")
    print("-" * 50)
    print(summary)
    
    return {
        "scores": scores,
        "summary": summary
    }

# Run the complete analysis
test_resume = """
Full-stack developer with 3 years of experience building web applications.
Strong background in React, Node.js, and PostgreSQL. Led the development of
customer-facing features used by 50K+ users. Collaborated with product team
to prioritize features and meet aggressive deadlines.
"""

result = analyze_resume(test_resume)

## Summary

This notebook demonstrated:

1. **RoleColor Keyword Framework** - Weighted keyword mappings for Builder, Enabler, Thriver, and Supportee
2. **Resume Scoring** - Algorithm to score resumes and identify dominant RoleColor
3. **Summary Rewriting** - AI-powered summary generation aligned to RoleColor traits

For the full interactive experience with LaTeX generation, E2B compilation, and chat assistance, run:

```bash
streamlit run app.py
```